# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 4. Embeddings
**모델링 과정**
1. 사전 학습된 모델을 Sentence Transformer 를 통해 불러오기
2. 수집하고 기본적인 전처리를 거친 데이터를 위의 모델을 통해 임베딩 벡터로 변환한 파생변수 생성하기
3. input 으로 사용자가 재료를 포함한 문자열을 입력하면 문장을 벡터화하여 기존의 임베딩 벡터와 코사인 유사도를 구하는 함수
4. 코사인 유사도 상위 기준으로 n개를 output 으로 추천
5. OpenAI 의 API 를 활용하여 사용자의 문장 형태 input 의 의도를 파악하여 모델링 함수를 실행하도록 연동
6. streamlit 에 연동하여 배포


### 라이브러리 import

In [2]:
# 데이터 분석
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 진행시간 표시
import swifter
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
# 파이토치
import torch

# 문장 임베딩, transformer 유틸리티
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import SentenceTransformer, models

In [4]:
# 객체 복사
import copy

# JSON 형식 데이터 처리
import json

# 데이터 수집
import requests

In [5]:
# 데이터베이스 활용
import sqlite3 
import pickle

In [6]:
# OpenAI API 활용
import openai 
import os # 운영체제
import sys # 파이썬 변수, 함수 엑세스 
from dotenv import load_dotenv # 환경 변수 로드(API Key 보안)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()    
openai.api_key = os.getenv('OPENAI_API_KEY')

In [7]:
# 실행 os 확인
cur_os = sys.platform
cur_os

'win32'

### 데이터 불러오기

In [8]:
df = pd.read_csv('data/preprocessed_recipes.csv')
df.shape

(4947, 11)

In [9]:
df.head(1)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수,질병
0,올리브 소시지 솥밥,메인요리,1,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10,일반


In [10]:
difficulty_map = {1: '쉬움', 2: '보통', 3: '어려움'}
df['난이도'] = df['난이도'].map(difficulty_map)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4947 entries, 0 to 4946
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요리      4947 non-null   object
 1   종류      4947 non-null   object
 2   난이도     4947 non-null   object
 3   소요시간    4947 non-null   int64 
 4   링크      4947 non-null   object
 5   사진      4947 non-null   object
 6   재료      4947 non-null   object
 7   요리방법    4947 non-null   object
 8   설명      4947 non-null   object
 9   재료수     4947 non-null   int64 
 10  질병      4947 non-null   object
dtypes: int64(2), object(9)
memory usage: 425.3+ KB


### 전이학습을 통한 임베딩 벡터 생성
- 사전학습 모델을 활용
- feature 생성 및 비교

### feature 생성

In [12]:
df['feature'] = df['재료'] + df['요리'] + df['설명'] + df['종류'] + df['난이도'] + df['요리방법'] + df['질병']
df['feature']

0       소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯올리브...
1       파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼원 팟 파스타냄...
2       브리치즈 마늘 올리브오일 바게트 꿀 로즈마리 후춧가루 방울토마토그릴드 브리치즈브리치...
3       소금 루꼴라 복숭아 올리브오일 화이트발사믹 민트잎 꿀 후춧가루 방울토마토 부라타치즈...
4       소금 양파 올리브오일 아보카도 고수 토마토 토르티야 슈파마산치즈 레몬과카몰리 부리또...
                              ...                        
4942    설탕 골뱅이통조림 물 튀김가루 간장 식초 고춧가루 식용유 대파골뱅이튀김과 파채특유의...
4943    다진파슬리 다진마늘 청주 피자치즈 대하 후춧가루 마요네즈 레몬대하치즈구이비주얼부터 ...
4944    소금 타임 올리브오일 아스파라거스 녹인버터 새송이버섯 로즈마리 후춧가루 편마늘 방울...
4945    다진파슬리 달걀 피자치즈 단호박 슬라이스햄단호박 에그슬럿노란빛의 달콤한 속살을 가진...
4946    소금 타임 디종머스타드 코코넛밀크 후춧가루 두유 뉴트리셔널이스트 파슬리 대파대파그라...
Name: feature, Length: 4947, dtype: object

In [13]:
df.to_csv('data/feature_recipes.csv', index=False)

### 사전 학습 모델
- model : Sentence Transformer
- pre trained model : `jhgan/ko-sroberta-multitask`

In [14]:
model_name = 'jhgan/ko-sroberta-multitask'

In [15]:
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [16]:
# feature1
feature_name = 'feature'
df[f'{model_name}-' + f'{feature_name}'] = df['feature'].progress_apply(lambda x: model.encode(x))

  0%|          | 0/4947 [00:00<?, ?it/s]

In [17]:
len(df.columns)

13

In [19]:
df.columns = ['요리', '종류', '난이도', '소요시간', '링크', '사진', '재료', '요리방법', '설명', '재료수', '질병', 'feature', 'ko-sroberta-multitask-feature']

In [20]:
df = df[['요리', '종류', '난이도', '재료', '요리방법', '설명', '재료수', '소요시간', '링크', '사진', '질병',
         'feature', 'ko-sroberta-multitask-feature']]
df

,요리,종류,난이도,재료,요리방법,설명,재료수,소요시간,링크,사진,질병,feature,ko-sroberta-multitask-feature
0,올리브 소시지 솥밥,메인요리,쉬움,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,일반,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯올리브...,"[0.088191085, -0.08414504, -0.14284688, -0.264..."
1,원 팟 파스타,메인요리,쉬움,파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼,"['방울토마토는 반으로 썰어주세요.\xa0파슬리는 다져주세요.', '냄비에 파스타면...","냄비에 파스타와 토마토, 올리브 등 재료를 넣고 물을 부은 후 끓이면 완성되는 초간...",9,20,https://wtable.co.kr/recipes/EzDURnKruCpR2nGxZ...,https://static.wtable.co.kr/image/production/s...,일반,파스타면 소금 올리브오일 물 후춧가루 파슬리 올리브 방울토마토 케이퍼원 팟 파스타냄...,"[-0.053257108, 0.070922054, -0.02161145, -0.61..."
2,그릴드 브리치즈,메인요리,쉬움,브리치즈 마늘 올리브오일 바게트 꿀 로즈마리 후춧가루 방울토마토,"['브리치즈에 칼집을 넣어주세요. 마늘은 편을 썰어주세요.\xa0', '오븐용기에 ...","브리치즈와 토마토, 오일과 허브를 뿌린 후 오븐에 구워내 보세요. 와인안주로 활용하...",8,30,https://wtable.co.kr/recipes/Tzr7zXZuJvm35s1wd...,https://static.wtable.co.kr/image/production/s...,일반,브리치즈 마늘 올리브오일 바게트 꿀 로즈마리 후춧가루 방울토마토그릴드 브리치즈브리치...,"[-0.071512364, -0.16087213, -0.052535467, 0.07..."
3,그릴드 피치 샐러드,메인요리,쉬움,소금 루꼴라 복숭아 올리브오일 화이트발사믹 민트잎 꿀 후춧가루 방울토마토 부라타치즈,['복숭아는 씨를 빼고 웨지 모양으로 썰어주세요. 작은 것은 반으로 썰어 주세요.\...,복숭아에 멋진 그릴 자국을 내보세요. 여름을 맞아 제철인 단단한 복숭아를 그릴에 구...,10,25,https://wtable.co.kr/recipes/RofraF8aFP5u5rqeX...,https://static.wtable.co.kr/image/production/s...,일반,소금 루꼴라 복숭아 올리브오일 화이트발사믹 민트잎 꿀 후춧가루 방울토마토 부라타치즈...,"[-0.070178695, 0.05069192, -0.26207393, 0.2856..."
4,과카몰리 부리또콘,메인요리,쉬움,소금 양파 올리브오일 아보카도 고수 토마토 토르티야 슈파마산치즈 레몬,['또띠아를 반으로 자른 후 원 뿔 모양으로 둥글게 말아 나무 꼬치로 가장자리를 고...,아보카도는 불포화 지방산이 풍부해 콜레스테롤 수치를 낮추고 혈관건강에 유익해요. 게...,9,20,https://wtable.co.kr/recipes/3PW95soZHRArnzsYB...,https://static.wtable.co.kr/image/production/s...,일반,소금 양파 올리브오일 아보카도 고수 토마토 토르티야 슈파마산치즈 레몬과카몰리 부리또...,"[-0.0049587316, 0.069201306, -0.17497921, 0.61..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4942,골뱅이튀김과 파채,오븐 요리,보통,설탕 골뱅이통조림 물 튀김가루 간장 식초 고춧가루 식용유 대파,['골뱅이는 물기를 제거한 후 튀김가루를 묻혀주세요. 튀김반죽 재료를 넣고 섞은 후...,"특유의 쫄깃한 식감이 매력적인 골뱅이. 지금까지 골뱅이무침만 먹어 오셨다면, 오늘은...",9,25,https://wtable.co.kr/recipes/8afBworBmh46MEuuM...,https://static.wtable.co.kr/image/production/s...,일반,설탕 골뱅이통조림 물 튀김가루 간장 식초 고춧가루 식용유 대파골뱅이튀김과 파채특유의...,"[-0.27840194, 0.03972874, 0.09049826, 0.318712..."
4943,대하치즈구이,오븐 요리,보통,다진파슬리 다진마늘 청주 피자치즈 대하 후춧가루 마요네즈 레몬,['대하의 수염과 다리를 잘라 손질해 주세요. 등을 따라 껍데기에 가위집을 내고 칼...,비주얼부터 감탄에 감탄을 부르는 대하치즈구이! 탱글탱글한 대하를 동그랗게 펼친 후 ...,8,30,https://wtable.co.kr/recipes/fgVuSyhNShABoMvcd...,https://static.wtable.co.kr/image/production/s...,일반,다진파슬리 다진마늘 청주 피자치즈 대하 후춧가루 마요네즈 레몬대하치즈구이비주얼부터 ...,"[0.03511691, 0.22847946, 0.34631813, 0.2503743..."
4944,돈마호크,오븐 요리,보통,소금 타임 올리브오일 아스파라거스 녹인버터 새송이버섯 로즈마리 후춧가루 편마늘 방울...,"['돈마호크를 밑간 재료에 10분간 재워주세요.', '법랑 접시에 석쇠를 놓고 돈마...","소고기에 토마호크가 있다면, 돼지고기엔 돈마호크가 있죠. 돈마호크는 돼지고기의 삼겹...",11,30,https://wtable.co.kr/recipes/nZ2EMyCWvh8jo7Y9S...,https://static.wtable.co.kr/image/production/s...,일반,소금 타임 올리브오일 아스파라거스 녹인버터 새송이버섯 로즈마리 후춧가루 편마늘 방울...,"[0.22016503, -0.15433419, -0.065425135, 0.0733..."
4945,단호박 에그슬럿,오븐 요리,보통,다진파슬리 달걀 피자치즈 단호박 슬라이스햄,['법랑 접시에 석쇠를 놓고 깨끗이 씻은 단호박을 올려 1단에 넣고 광파오븐 수동 ...,노란빛의 달콤한 속살을 가진 단호박은 식이섬유가 많고 열량이 낮아 다이어트에 효과적...,5,30,https://wtable.co.kr/recipes/JERayqDgKrLf4fawy...,https://static.wtable.co.kr/image/production/s...,비만,다진파슬리 달걀 피자치즈 단호박 슬라이스햄단호박 에그슬럿노란빛의 달콤한 속살을 가진...,"[0.08519404, -0.30381483, 0.46431497, 0.184560..."


In [21]:
df.to_pickle('data/compact_kosroberta_recipes.pkl')
df = pd.read_pickle('data/compact_kosroberta_recipes.pkl')

In [22]:
df.head(1)

,요리,종류,난이도,재료,요리방법,설명,재료수,소요시간,링크,사진,질병,feature,ko-sroberta-multitask-feature
0,올리브 소시지 솥밥,메인요리,쉬움,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯,"['쌀은 씻어 채에 받쳐 30분 정도 불려주세요.\xa0', '소세지는 모양대로 슬...",올리브를 넣어 이국적인 감칠맛이 매력적인 솥 밥. 별다른 양념없이도 술술 넘어가는 ...,10,30,https://wtable.co.kr/recipes/rF7F5gmLrySsw7ZXZ...,https://static.wtable.co.kr/image/production/s...,일반,소시지 올리브오일 쌀 치킨스톡파우더 물 후춧가루 파마산치즈 파슬리 올리브 샬롯올리브...,"[0.088191085, -0.08414504, -0.14284688, -0.264..."


## DB에 임베딩 벡터 저장하고, 불러오는 과정
- 위에서 적용한 pickle 형식으로 데이터를 저장하는 것이 더 효율적이라고 판단됨

In [23]:
# # 데이터 프레임의 embedding
# def save_embeddings_to_db(df, db_file, model):
#     conn = sqlite3.connect(db_file)
#     cursor = conn.cursor()
#     cursor.execute('''CREATE TABLE IF NOT EXISTS embedding_table (id INTEGER PRIMARY KEY, embedding_data BLOB)''')
#     conn.commit()

#     for index, row in df.iterrows():
#         text = row['feature']
#         embedding = model.encode(text)
#         embedding_binary = pickle.dumps(embedding)
#         cursor.execute('''INSERT INTO embedding_table (id, embedding_data) VALUES (?,?)''', (index, embedding_binary))
#         conn.commit()

#     conn.close()

In [24]:
# save_embeddings_to_db(df, 'embedding.db', model)

In [25]:
# def add_embeddings_to_df(df, db_file):
#     conn = sqlite3.connect(db_file)
#     cursor = conn.cursor()

#     # 데이터베이스에서 임베딩 정보 불러오기
#     cursor.execute('''SELECT id, embedding_data FROM embedding_table''')
#     rows = cursor.fetchall()
    
#     # 불러온 임베딩 정보를 딕셔너리로 변환
#     embeddings_dict = {}
#     for row in rows:
#         index = row[0]
#         embedding_binary = row[1]
#         embedding = pickle.loads(embedding_binary)
#         embeddings_dict[index] = embedding
    
#     # 데이터프레임에 임베딩 정보 추가
#     embeddings = []
#     for index, row in df.iterrows():
#         if index in embeddings_dict:
#             embedding = embeddings_dict[index]
#             embeddings.append(embedding)
#         else:
#             embeddings.append(None)
#     df['embeddings'] = embeddings
    
#     conn.close()
#     return df


In [26]:
# add_embeddings_to_df(df, 'embedding.db')
# df